In [2]:
from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences
import numpy as np

In [3]:
maxlen = 10
#training_samples = 20
#validation_samples = 100
max_words = 10

In [4]:
Model_tokenizer = Tokenizer(num_words = max_words)

In [5]:
texts = ["This is a girl.","Girl is tall","A tall boy is here",]

In [6]:
#model training
Model_tokenizer.fit_on_texts(texts)

In [7]:
#use the trained model for predicting the sequence number
sequences = Model_tokenizer.texts_to_sequences(texts)

In [8]:
sequences

[[5, 1, 2, 3], [3, 1, 4], [2, 4, 6, 1, 7]]

In [9]:
text_new = ["this is my house","the girl and boy","The house is small"]

In [10]:
sequences_new = Model_tokenizer.texts_to_sequences(text_new)
print(sequences_new)

[[5, 1], [3, 6], [1]]


In [11]:
seq_try = [[5, 1,2,23], [3, 6]]
text_try = Model_tokenizer.sequences_to_texts(seq_try)
print(text_try)

['this is a', 'girl boy']


In [12]:
print(Model_tokenizer.document_count)
print(Model_tokenizer.get_config())

3
{'num_words': 10, 'filters': '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', 'lower': True, 'split': ' ', 'char_level': False, 'oov_token': None, 'document_count': 3, 'word_counts': '{"this": 1, "is": 3, "a": 2, "girl": 2, "tall": 2, "boy": 1, "here": 1}', 'word_docs': '{"girl": 2, "this": 1, "a": 2, "is": 3, "tall": 2, "here": 1, "boy": 1}', 'index_docs': '{"3": 2, "5": 1, "2": 2, "1": 3, "4": 2, "7": 1, "6": 1}', 'index_word': '{"1": "is", "2": "a", "3": "girl", "4": "tall", "5": "this", "6": "boy", "7": "here"}', 'word_index': '{"is": 1, "a": 2, "girl": 3, "tall": 4, "this": 5, "boy": 6, "here": 7}'}


In [13]:
data = pad_sequences(sequences,maxlen=6)
print(data)

[[0 0 5 1 2 3]
 [0 0 0 3 1 4]
 [0 2 4 6 1 7]]


In [14]:
data_new = pad_sequences(sequences_new,maxlen=4)
print(data_new)

[[0 0 5 1]
 [0 0 3 6]
 [0 0 0 1]]


## Text classification with TensorFlow Hub: Movie reviews

In [15]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [16]:
# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteT74HK3/imdb_reviews-train.tfrecord…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteT74HK3/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteT74HK3/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [17]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(4)))
train_examples_batch

<tf.Tensor: shape=(4,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell a

In [18]:
train_labels_batch

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([0, 0, 0, 1])>

In [19]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:4])

<tf.Tensor: shape=(4, 50), dtype=float32, numpy=
array([[ 5.42319477e-01, -1.19017037e-02,  6.33753762e-02,
         6.86297193e-02, -1.67768374e-01, -1.05811745e-01,
         1.68653026e-01, -4.99882400e-02, -3.11480552e-01,
         7.91034624e-02,  1.54422626e-01,  1.48866158e-02,
         3.93015295e-02,  1.97727114e-01, -1.22154757e-01,
        -4.12098095e-02, -2.70410895e-01, -2.19221517e-01,
         2.65176624e-01, -8.07390749e-01,  2.58335322e-01,
        -3.10042113e-01,  2.86832154e-01,  1.94338694e-01,
        -2.90364921e-01,  3.86284851e-02, -7.84441113e-01,
        -4.79324013e-02,  4.11029905e-01, -3.63888919e-01,
        -5.80347061e-01,  3.02694559e-01,  3.63089710e-01,
        -1.52271643e-01, -4.43915039e-01,  1.94629967e-01,
         1.95284083e-01,  5.66623397e-02,  2.89070398e-01,
        -2.84683228e-01, -5.31205582e-03,  5.71938045e-02,
        -3.20131809e-01, -4.41866480e-02, -8.55078250e-02,
        -5.58474362e-01, -2.33363912e-01, -2.07829520e-01,
       

In [20]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [22]:
len(train_data)

15000

In [23]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
30/30 [==============================] - 41s 1s/step - loss: 0.6488 - accuracy: 0.5324 - val_loss: 0.5966 - val_accuracy: 0.6020
Epoch 2/10
30/30 [==============================] - 39s 1s/step - loss: 0.5259 - accuracy: 0.7090 - val_loss: 0.4831 - val_accuracy: 0.7422
Epoch 3/10
30/30 [==============================] - 40s 1s/step - loss: 0.3871 - accuracy: 0.8306 - val_loss: 0.3866 - val_accuracy: 0.8291
Epoch 4/10
30/30 [==============================] - 41s 1s/step - loss: 0.2735 - accuracy: 0.8935 - val_loss: 0.3381 - val_accuracy: 0.8479
Epoch 5/10
30/30 [==============================] - 41s 1s/step - loss: 0.1977 - accuracy: 0.9301 - val_loss: 0.3168 - val_accuracy: 0.8617
Epoch 6/10
30/30 [==============================] - 40s 1s/step - loss: 0.1440 - accuracy: 0.9558 - val_loss: 0.3099 - val_accuracy: 0.8666
Epoch 7/10
30/30 [==============================] - 40s 1s/step - loss: 0.1049 - accuracy: 0.9706 - val_loss: 0.3130 - val_accuracy: 0.8717
Epoch 8/10
30/30 [==

In [24]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 16s - loss: 0.3659 - accuracy: 0.8529 - 16s/epoch - 336ms/step
loss: 0.366
accuracy: 0.853
